In [1]:
import torch
import torch.nn as nn
from spikingjelly.activation_based import functional,neuron,layer
import numpy as np

/home/udayanga/anaconda3/envs/SNNTEST/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.data import Dataset
import os
import torchvision.transforms as Tr

In [3]:
from collections import OrderedDict

In [6]:
import pytorch_lightning as pl


ModuleNotFoundError: No module named 'yaml'

In [1]:
import pytorch_lightning as pl
import torchmetrics
from torch.utils.data import DataLoader

/home/udayanga/anaconda3/envs/SNNTEST/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'yaml'

In [5]:
import h5py

In [6]:
cfgs = {'A' : [64,'M',128,'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
         'B' : [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'A']}

In [7]:
class VGGSNN(nn.Module):
    def __init__(self,num_of_inChannels,cfg,norm_layer=None,num_classes=4,init_weights=True,single_step_neuron:callable = None,**kwargs):
        super(VGGSNN,self).__init__()

        self.out_channels = []
        self.idx_pool = [i for i,v in enumerate(cfg) if v=='M']
        if norm_layer is None:
            norm_layer = nn.Identity
        bias = isinstance(norm_layer,nn.Identity)

        self.features = self.make_layers(num_of_inChannels,cfg=cfg,norm_layer=norm_layer,lifNeuron=single_step_neuron,bias=bias,**kwargs)

        self.classifier = nn.Sequential(OrderedDict([("norm_classif",norm_layer(cfg[-2])),
                                                        ("conv_classif",nn.Conv2d(cfg[-2],num_classes,kernel_size=1,bias=bias))
                                                            ])
                                                            )
        if init_weights:
            self._initialize_weights()
    
    def forward(self,x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    def make_layers(self,num_of_inChannels,cfg,norm_layer,lifNeuron,bias,**kwargs):
        layers = []
        channel_in_info = num_of_inChannels
        for v in cfg:
            if v == 'M':
                layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
                self.out_channels.append(channel_in_info)
            else:
                #this may be unwrapped later        
                layers.append(nn.ConstantPad2d(padding=(1, 1, 1, 1), value=0.0))
                layers.append(norm_layer(channel_in_info))
                layers.append(nn.Conv2d(channel_in_info, v, kernel_size=(3, 3), stride=(1, 1), bias=False))
                layers.append(lifNeuron(**kwargs))
                channel_in_info = v
                
        self.out_channels = self.out_channels[2:]
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def add_hooks(self):
        def get_nz(name):
            def hook(model, input, output):
                self.nz[name] += torch.count_nonzero(output)
                self.numel[name] += output.numel()
            return hook
        
        self.hooks = {}
        for name, module in self.named_modules():
            self.nz[name], self.numel[name] = 0, 0
            self.hooks[name] = module.register_forward_hook(get_nz(name))
                
    def reset_nz_numel(self):
        for name, module in self.named_modules():
            self.nz[name], self.numel[name] = 0, 0
        
    def get_nz_numel(self):
        return self.nz, self.numel

In [8]:
class MultiStepVGGSNN(VGGSNN):
    def __init__(self, num_of_input_channels, cfg, norm_layer=None, num_classes=4, init_weights=True, timeSteps : int = None,
                 multi_step_neuron: callable = None, **kwargs):
        self.TimeSteps = timeSteps
        super().__init__(num_of_input_channels, cfg, norm_layer, num_classes, init_weights,
                 multi_step_neuron, **kwargs)
                 
    def forward(self,x,classify = True):
        x_seq = x
        if x.dim() != 5:
            assert self.T is not None, 'When x.shape is [N, C, H, W], self.T can not be None.'
        
        if classify:
            #x_seq = functional.seq_to_ann_forward(x, self.features[0])
            z_seq_step_by_step = []
            for t in range(self.TimeSteps):
                x = x_seq[t]
                y = self.features(x)
                z = self.classifier(y)
                z_seq_step_by_step.append(z.unsqueeze(0))
            z_seq_step_by_step = torch.cat(z_seq_step_by_step, 0)
            z_seq_step_by_step = z_seq_step_by_step.flatten(start_dim=-2).sum(-1)
            return z_seq_step_by_step

In [9]:
class ClassificationDataset(Dataset):
    def __init__(self, mode,image_shape,dataSet_name,file_path,numOfTimeSteps = 5,sampleSize = 97620,numOfTbins = 2):
        self.mode = mode #whether test or train
        self.tbin = numOfTbins
        self.C, self.T = self.tbin * 2, numOfTimeSteps
        self.sample_size = sampleSize
        self.quantization_size = [self.sample_size//self.T,1,1] #sample length for one time step
        self.w, self.h = image_shape
        self.quantized_w = self.w // self.quantization_size[1]
        self.quantized_h = self.h // self.quantization_size[2]
        
        save_file_name = f"{dataSet_name}_{self.mode}_{self.sample_size//1000}_{self.quantization_size[0]/1000}ms_{self.tbin}tbin.pt"
        save_file = os.path.join(file_path, save_file_name)
        
        if os.path.isfile(save_file):
            self.samples = torch.load(save_file)
            print("File loaded.")
        else:
            data_dir = os.path.join(file_path, mode)
            self.samples = self.build_dataset(data_dir, save_file)
            torch.save(self.samples, save_file)
            print(f"Done! File saved as {save_file}.")
            
    def __getitem__(self, index):

        coords, feats, target = self.samples[index]
        sample = torch.sparse_coo_tensor(coords.t(), feats.to(torch.float32)).coalesce()
        sample = sample.sparse_resize_(
            (self.T, sample.size(1), sample.size(2), self.C), 3, 1
        ).to_dense().permute(0,3,1,2)                     #(T,C,W,H)
        
        sample = Tr.Resize((64,64), Tr.InterpolationMode.NEAREST)(sample) 
        
        return sample, target

    def __len__(self):
        return len(self.samples)

    def build_dataset(self, data_dir, save_file):
        raise NotImplementedError("The method build_dataset has not been implemented.")

In [10]:
import h5py

In [11]:
def load_v2e_event_data(filename,augmentType):
    """Load V2E Events, all HDF5 records."""
    assert os.path.isfile(filename)
    event_seg_list = []
    v2e_data = h5py.File(filename, "r")
    events = v2e_data["events"][()]
    events_2 = events[events[:,0] < 50000]
    event_seg_list.append(events_2)
    if augmentType == 1: #healthy
        events_3 = events_2
        events_3[:,1] = 345 - events_3[:,1]
        event_seg_list.append(events_3)
    elif augmentType == 3: #corros
        events_3 = events_2
        events_3[:,1] = 345 - events_3[:,1]
        event_seg_list.append(events_3)
    elif augmentType == 4: #eff
        events_3 = events_2
        events_3[:,1] = 345 - events_3[:,1]
        events_4 = events_2
        events_4[:,2] = 259 - events_4[:,2]
        event_seg_list.append(events_3)
        event_seg_list.append(events_4)
    return event_seg_list

In [12]:
events = load_v2e_event_data("/home/udayanga/ResearchWork/SNN/selected_train_test/train/spalling/accept033070.h5")

TypeError: load_v2e_event_data() missing 1 required positional argument: 'augmentType'

In [ ]:
sss = events[:,0:3]
qqq = [97620//5,1,1]
ttt = torch.floor(sss/torch.tensor(qqq))
ttt
tbin_size = qqq[0]/ 2
print(tbin_size)
tbin_coords = (events[:,0] % qqq[0]) // tbin_size
np.max(tbin_coords)

NameError: name 'events' is not defined

In [12]:
class v2EDamageDataSet(ClassificationDataset):
    def __init__(self,mode,data_folder_path,image_shape = (346,260),dataSet_name = "v",numOfTimeSteps = 5,sampleSize = 97620,numOfTbins=2):
        super().__init__(mode,image_shape,dataSet_name,data_folder_path,numOfTimeSteps,sampleSize)
    
    def build_dataset(self,data_dir,save_file):
        print("building v2e damage dataset ....")
        classes_dir = [os.path.join(data_dir,class_name) for class_name in os.listdir(data_dir)]
        samples = []
        current_max = 0
        print("class dir ",classes_dir)
        for class_id,class_dir in enumerate(classes_dir):

            self.files = [os.path.join(class_dir,time_seq_name) for time_seq_name in os.listdir(class_dir)]
            target = class_id
            print(f'Building the class id {class_id} and class dir {class_dir}')
            
            for file_name in self.files:
                #print("processing ",file_name)
                events_seg_list = load_v2e_event_data(file_name,class_id)
                for events in events_seg_list:
                    #print("printing events ...",events)
                    #print("events shape ",events.shape)
                    print("number of events",len(events))
                    if(events.size == 0):
                        print("Empty sample ....", file_name, "for class",class_id)
                        continue
                    if (np.sum(events[events[:,3] > 1]) > 1):
                        print("got a large value ",events[:,3])
                        break

                    # Bin the events on T timesteps
                    coords = events[:,0:3]
                    #print("coords 1 0 ",coords)
                    coords = torch.floor(coords/torch.tensor(self.quantization_size)) 
                    coords[:, 1].clamp_(min=0, max=self.quantized_w-1)
                    coords[:, 2].clamp_(min=0, max=self.quantized_h-1)
                    #print("coords 2 = ",coords)
                    if (coords[:,0].max() > 4):
                        print("exceeding time bins to ",coords[:,0].max()," ",file_name)
                    # TBIN computations
                    #else:
                    #    print("max time ",coords[:,0].max())
                    tbin_size = self.quantization_size[0] / self.tbin
                    tbin_coords = (events[:,0] % self.quantization_size[0]) // tbin_size
                    tbin_feats = (2* tbin_coords) + events[:,3]
                    feats = torch.nn.functional.one_hot(torch.from_numpy(tbin_feats).to(torch.long), 2*self.tbin).to(bool)
                    
                    samples.append([coords.to(torch.int16), feats.to(bool), target])
        return samples
                
    

In [13]:
folder_path_for_event_data  = "/home/udayanga/ResearchWork/SNN/selected_train_test/"

In [14]:
trainDataSet = v2EDamageDataSet("train",folder_path_for_event_data,numOfTimeSteps = 5,sampleSize = 50000,numOfTbins = 2)

File loaded.


In [ ]:
#import torch

/home/udayanga/anaconda3/envs/SNNTEST/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
"""class tempModule(pl.LightningModule):
    
    def __init__(self, model, epochs=10, lr=5e-3, num_classes=4):
        super().__init__()
        self.train_acc = torchmetrics.Accuracy(num_classes=num_classes)
        self.train_acc_by_class = torchmetrics.Accuracy(num_classes=num_classes,average='none')
    def temp(self,mode):
        acc = getattr(self, f"{mode}_acc")
        acc_by_class = getattr(self, f"{mode}_acc_by_class")
        sm_outputs = torch.tensor([[0.25,0.1,0.25,0.4],[0.25,0.1,0.25,0.4],[0.25,0.1,0.25,0.4],[0.25,0.1,0.25,0.4]])
        target = torch.tensor([3,2,3,2])
        print(target)
        #print(loss)
        acc(sm_outputs, target)
        acc_by_class(sm_outputs, target)
        acc_val = acc.compute()
        acc_by_class_val = acc_by_class.compute()
        print(f"accuracy: {100*acc_val:.2f}%")
        print(acc_by_class_val[0])
        print(f"spalling {100*acc_by_class_val[0]:.2f}% - healthy {100*acc_by_class_val[3]:.2f}% - bbb {100*acc_by_class_val[2]:.2f}%")"""

In [ ]:
"""bb = tempModule(None,2,0.02,4)
bb.temp("train")
x = torch.FloatTensor([[5.,0.,6.0],
                       [5.,0.,0.],
                       [1.,0.,5.]])
y = torch.LongTensor([0,1,2])

print(torch.nn.functional.cross_entropy(x, y))
from torchmetrics import Accuracy

w = Accuracy(4)
w(torch.tensor([0.2]),torch.tensor([]))
"""

tensor(2.1177)


In [ ]:
class ClassificationLitModule(pl.LightningModule):
    def __init__(self, model, epochs=10, lr=5e-3, num_classes=4):
        super().__init__()
        #self.save_hyperparameters()
        self.lr, self.epochs = lr, epochs
        self.num_classes = num_classes

        self.model = model

        self.train_acc = torchmetrics.Accuracy(num_classes=num_classes)
        self.val_acc = torchmetrics.Accuracy(num_classes=num_classes)
        self.test_acc = torchmetrics.Accuracy(num_classes=num_classes)
        self.train_acc_by_class = torchmetrics.Accuracy(num_classes=num_classes, average="none")
        self.val_acc_by_class = torchmetrics.Accuracy(num_classes=num_classes, average="none")
        self.test_acc_by_class = torchmetrics.Accuracy(num_classes=num_classes, average="none")
        self.train_confmat = torchmetrics.ConfusionMatrix(num_classes=num_classes)
        self.val_confmat = torchmetrics.ConfusionMatrix(num_classes=num_classes)
        self.test_confmat = torchmetrics.ConfusionMatrix(num_classes=num_classes)
        
    def forward(self, x):
        
        x = x.permute(1,0,2,3,4)
        
        retVal = self.model(x)
        print("input shape for the classLit Forward ",x.shape)
        print("ClassLit return value shape = ",retVal.shape)
        #retVal = self.model(x).sum(dim=1)
        retVal = retVal.permute(1,0,2)
        retVal = retVal.sum(dim=1)
        return retVal
    
    def step(self, batch, batch_idx, mode):
        events, target = batch
        #print("batch_shape  ",batch[0].shape)
        #print("batch target shape  ",batch[1].shape)
        #print("printing target ",target)
        outputs = self(events)
        #print("eventssssssss = ", events.shape)
        #print("outputsttttttttt ",outputs)
        loss = nn.functional.cross_entropy(outputs, target)

        # Measure sparsity if testing
        #if mode=="test":
        #    self.process_nz(self.model.get_nz_numel())

        # Metrics computation
        sm_outputs = outputs.softmax(dim=-1)
        print("printing softmax outputs =",sm_outputs)

        acc, acc_by_class = getattr(self, f"{mode}_acc"), getattr(self, f"{mode}_acc_by_class")
        confmat = getattr(self, f'{mode}_confmat')
       
        #print("target as one hot encoded =",target.shape)
        #print("target shape 22222 ",)
        acc(sm_outputs, target)
        acc_by_class(sm_outputs, target)
        confmat(sm_outputs, target)

        if mode != "test":
            self.log(f'{mode}_loss', loss, on_epoch=True, prog_bar=(mode == "train"))
        if mode == "test":
            mode_acc, mode_acc_by_class = getattr(self, f"{mode}_acc"), getattr(self, f"{mode}_acc_by_class")
            acc, acc_by_class = mode_acc.compute(), mode_acc_by_class.compute()
            for i,acc_i in enumerate(acc_by_class):
                self.log(f'{mode}_acc_{i}', acc_i)
            self.log(f'{mode}_acc', acc)
            mode_acc.reset()
            mode_acc_by_class.reset()

        print(f"accuracy: {100*acc:.2f}%")
        print(f"spalling {100*acc_by_class[0]:.2f}% - healthy {100*acc_by_class[1]:.2f}% - crack {100*acc_by_class[2]:.2f}% -  efflores {100*acc_by_class[3]:.2f}%")
        functional.reset_net(self.model)
        return loss
        
    def training_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, mode="train")

    def validation_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, mode="val")

    def test_step(self, batch, batch_idx):
        return self.step(batch, batch_idx, mode="test")

    def on_mode_epoch_end(self, mode):
        print()
        mode_acc, mode_acc_by_class = getattr(self, f"{mode}_acc"), getattr(self, f"{mode}_acc_by_class")
        acc, acc_by_class = mode_acc.compute(), mode_acc_by_class.compute()
        for i,acc_i in enumerate(acc_by_class):
            self.log(f'{mode}_acc_{i}', acc_i)
        self.log(f'{mode}_acc', acc)

        print(f"accuracy: {100*acc:.2f}%")
        print(f"spalling {100*acc_by_class[0]:.2f}% - healthy {100*acc_by_class[1]:.2f}% - crack {100*acc_by_class[1]:.2f}% - corrosion {100*acc_by_class[1]:.2f}% - efflore {100*acc_by_class[1]:.2f}%")
        #print(f"{mode} accuracy: {100*acc:.2f}%")
        #print(f"spalling {100*acc_by_class[0]:.2f}% - healthy {100*acc_by_class[1]:.2f}% - crack {100*acc_by_class[2]:.2f}%  - corros {100*acc_by_class[2]:.2f}% -  efflores {100*acc_by_class[3]:.2f}%")
        mode_acc.reset()
        mode_acc_by_class.reset()
        """
        print(f"{mode} confusion matrix:")
        self_confmat = getattr(self, f"{mode}_confmat")
        confmat = self_confmat.compute()
        self.log(f'{mode}_confmat', confmat.mean())
        print(confmat.mean())
        self_confmat.reset()"""

        if mode=="test":
            print(f"Total sparsity: {self.all_nnz} / {self.all_nnumel} ({100 * self.all_nnz / self.all_nnumel:.2f}%)")
            self.all_nnz, self.all_nnumel = 0, 0

    def process_nz(self, nz_numel):
        nz, numel = nz_numel
        total_nnz, total_nnumel = 0, 0

        for module, nnz in nz.items():
            if "act" in module:
                nnumel = numel[module]
                if nnumel != 0:
                    total_nnz += nnz
                    total_nnumel += nnumel
        if total_nnumel != 0:
            self.all_nnz += total_nnz
            self.all_nnumel += total_nnumel

    def on_train_epoch_end(self):
        return self.on_mode_epoch_end(mode="train")

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr) #AdamW
        """scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            self.epochs,
        )"""
        return {"optimizer": optimizer}# "lr_scheduler": scheduler}

In [ ]:
train_dataloader = DataLoader(trainDataSet, batch_size=32, num_workers=8, shuffle=True)

In [ ]:
testDataSet = v2EDamageDataSet("test",folder_path_for_event_data,numOfTimeSteps = 5,sampleSize = 50000,numOfTbins = 2)

File loaded.


In [ ]:
test_dataloader = DataLoader(testDataSet, batch_size=8, num_workers=8)

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

In [ ]:
from pytorch_lightning.loggers import CometLogger

In [ ]:
callbacks=[]
ckpt_callback = ModelCheckpoint(
        monitor='train_loss',
        dirpath=f"ckpt-damage-classifier-vgg/",
        filename=f"damage-classifier-vgg" + "-{epoch:02d}-{train_acc:.4f}",
        save_top_k=3,
        mode='min',
    )
callbacks.append(ckpt_callback)

In [ ]:
logger = None
try:
    comet_logger = CometLogger(
        api_key=None,
        project_name=f"classif-damage-classifier-vgg/",
        save_dir="comet_logs",
        log_code=True,
        )
    logger = comet_logger
except ImportError:
    print("Comet is not installed, Comet logger will not be available.")


Comet is not installed, Comet logger will not be available.


In [ ]:
trainer = pl.Trainer(
        accelerator='gpu', devices=[0], gradient_clip_val=1., max_epochs=10,
        limit_train_batches=1., limit_val_batches=1.,
        check_val_every_n_epoch=1,
        deterministic=False,
        precision=16,
        callbacks=callbacks,
        logger=logger,
    )

Using 16bit native Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..


In [ ]:
norm_lyr = nn.BatchNorm2d
ms_neuron = neuron.ParametricLIFNode #step_mode='m',backend='cupy') #accelerate the processing in GPU with cupy
channels_per_timeStep = 4
number_of_classes = 5
model = MultiStepVGGSNN(channels_per_timeStep,cfg = cfgs['A'],norm_layer=norm_lyr,num_classes=number_of_classes,init_weights=True,timeSteps=5,multi_step_neuron = ms_neuron,step_mode='s')
module = ClassificationLitModule(model, epochs=5, lr=5e-3,num_classes=number_of_classes)

In [ ]:
trainer.fit(module,train_dataloader,test_dataloader)

/home/udayanga/anaconda3/envs/SNNTEST/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/udayanga/ResearchWork/SNN/ckpt-damage-classifier-vgg exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type            | Params
-------------------------------------------------------
0 | model              | MultiStepVGGSNN | 9.2 M 
1 | train_acc          | Accuracy        | 0     
2 | val_acc            | Accuracy        | 0     
3 | test_acc           | Accuracy        | 0     
4 | train_acc_by_class | Accuracy        | 0     
5 | val_acc_by_class   | Accuracy        | 0     
6 | test_acc_by_class  | Accuracy        | 0     
7 | train_confmat      | ConfusionMatrix | 0     
8 | val_confmat        | ConfusionMatrix | 0     
9 | test_confmat       | ConfusionMatrix | 0     
-------------------------------------

input shape for the classLit Forward  torch.Size([5, 8, 4, 64, 64])
ClassLit return value shape =  torch.Size([5, 8, 5])
printing softmax outputs = tensor([[0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]], device='cuda:0')


TypeError: unsupported format string passed to CompositionalMetric.__format__

In [ ]:
myPath = "/home/udayanga/ResearchWork/SNN/Synthesized_event_dataSets_2/Customized_for_event_gen/Event_Data_Collector/Only_events/NoDefect/"

In [ ]:
trainer.test(module,dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]input shape for the classLit Forward  torch.Size([5, 391, 4, 64, 64])
ClassLit return value shape =  torch.Size([5, 391, 4])
printing softmax outputs = tensor([[1.0000e+00, 5.1293e-08, 3.5609e-07, 2.6420e-07],
        [9.9952e-01, 3.7745e-04, 6.7574e-05, 3.1757e-05],
        [9.9938e-01, 1.4393e-05, 9.9964e-06, 5.9636e-04],
        ...,
        [4.3911e-01, 5.5254e-01, 8.0335e-03, 3.2239e-04],
        [9.8056e-01, 8.5836e-05, 1.9213e-02, 1.3914e-04],
        [6.9096e-06, 9.9969e-01, 2.9682e-04, 3.0321e-06]], device='cuda:0')
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.42it/s]


[{}]

In [ ]:
torch.

In [ ]:
import shutil

In [ ]:
dest_train_path = "/home/udayanga/ResearchWork/SNN/selected_train_test/train/healthy/"
numOfFilesToCopy = 402
dest_test_path = "/home/udayanga/ResearchWork/SNN/selected_train_test/test/healthy/"

In [ ]:
i = 0
for f1 in os.listdir(myPath):
    print(f1)
    tempPath = myPath + "/" + f1
    for f2 in os.listdir(tempPath):
        if (f2 == "_ideal"):
            tempPath2 = tempPath + "/" + f2
            for f3 in os.listdir(tempPath2):
            
                if f3.endswith("h5"):
                    fullSrcPath = tempPath2 + "/" + f3
                    print(fullSrcPath)
                if (i < numOfFilesToCopy):
                    shutil.copy(fullSrcPath,dest_train_path)
                else:
                    shutil.copy(fullSrcPath,dest_test_path)
                
                i += 1

NoDefect_287
/home/udayanga/ResearchWork/SNN/Synthesized_event_dataSets_2/Customized_for_event_gen/Event_Data_Collector/Only_events/NoDefect//NoDefect_287/_ideal/RIMG1525_ejagi2hg.4px.h5
NoDefect_371
/home/udayanga/ResearchWork/SNN/Synthesized_event_dataSets_2/Customized_for_event_gen/Event_Data_Collector/Only_events/NoDefect//NoDefect_371/_ideal/RIMG0813_zimezpo1.nxa.h5
NoDefect_154
/home/udayanga/ResearchWork/SNN/Synthesized_event_dataSets_2/Customized_for_event_gen/Event_Data_Collector/Only_events/NoDefect//NoDefect_154/_ideal/RIMG2151_qobpw1at.j5k.h5
NoDefect_304
/home/udayanga/ResearchWork/SNN/Synthesized_event_dataSets_2/Customized_for_event_gen/Event_Data_Collector/Only_events/NoDefect//NoDefect_304/_ideal/RIMG0785_b25rjx1i.hok.h5
NoDefect_146
/home/udayanga/ResearchWork/SNN/Synthesized_event_dataSets_2/Customized_for_event_gen/Event_Data_Collector/Only_events/NoDefect//NoDefect_146/_ideal/RIMG0743_31pc5jpx.4ap.h5
NoDefect_348
/home/udayanga/ResearchWork/SNN/Synthesized_event_da

In [ ]:
i = 0
for dirpath, dirnames, filenames in os.walk(myPath):
    
    for file in dirnames:
        print(file)
        i +=1

i  / 3
    

crack_local_421
crack_local_374
crack_local_95
crack_local_328
crack_local_270
crack_local_348
crack_local_480
crack_local_16
crack_local_259
crack_local_346
crack_local_454
crack_local_148
crack_local_417
crack_local_490
crack_local_301
crack_local_418
crack_local_94
crack_local_167
crack_local_306
crack_local_446
crack_local_308
crack_local_288
crack_local_2
crack_local_138
crack_local_122
crack_local_325
crack_local_445
crack_local_360
crack_local_59
crack_local_320
crack_local_223
crack_local_397
crack_local_198
crack_local_269
crack_local_453
crack_local_57
crack_local_30
crack_local_271
crack_local_196
crack_local_70
crack_local_89
crack_local_81
crack_local_260
crack_local_45
crack_local_145
crack_local_439
crack_local_361
crack_local_80
crack_local_239
crack_local_255
crack_local_180
crack_local_248
crack_local_272
crack_local_311
crack_local_62
crack_local_3
crack_local_256
crack_local_33
crack_local_205
crack_local_395
crack_local_174
crack_local_23
crack_local_246
crack_loca

661.3333333333334